In [19]:
import cv2

cap = cv2.VideoCapture('/opt/ml/input/data/test_video.mp4')
FPS = cap.get(5) 
frame = cap.get(7)
count = 1

for i in range(1, int(frame)):
    success, img = cap.read()
    
    if not success:
        break

    if i % (1.25) < 1:
        cv2.imwrite(f"/opt/ml/input/data/test_video/test_video"+'-'+str(count).zfill(6)+'.jpg', img)
        count+=1


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import videotransforms

import numpy as np
from pytorch_i3d import InceptionI3d
from charades_dataset_full import Charades as Dataset
from utils import createDirectory
import os

In [21]:
test_transforms = transforms.Compose([])
first_transforms = transforms.Compose([videotransforms.FirstCrop(224)])
second_transforms = transforms.Compose([videotransforms.SecondCrop(224)])
third_transforms = transforms.Compose([videotransforms.ThirdCrop(224)])
fourth_transforms = transforms.Compose([videotransforms.FourthCrop(224)])
center_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

In [22]:
split = '/opt/ml/input/data/test.json'
root = '/opt/ml/input/data'
mode = 'rgb'
batch_size = 1
save_dir = '/opt/ml/input/data'
load_model = '/opt/ml/input/code/pytorch-i3d/models/rgb_charades.pt'

first_dataset = Dataset(split, 'testing', root, mode, first_transforms, num=-1, save_dir=save_dir)
first_dataloader = torch.utils.data.DataLoader(first_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

second_dataset = Dataset(split, 'testing', root, mode, second_transforms, num=-1, save_dir=save_dir)
second_dataloader = torch.utils.data.DataLoader(second_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

third_dataset = Dataset(split, 'testing', root, mode, third_transforms, num=-1, save_dir=save_dir)
third_dataloader = torch.utils.data.DataLoader(third_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

fourth_dataset = Dataset(split, 'testing', root, mode, fourth_transforms, num=-1, save_dir=save_dir)
fourth_dataloader = torch.utils.data.DataLoader(fourth_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

center_dataset = Dataset(split, 'testing', root, mode, center_transforms, num=-1, save_dir=save_dir)
center_dataloader = torch.utils.data.DataLoader(center_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)


dataloaders = {
               '1': first_dataloader,
               '2': second_dataloader,
               '3': third_dataloader,
               '4': fourth_dataloader,
               '5': center_dataloader,
                }

datasets = {
            '1': first_dataset,
            '2': second_dataset,
            '3': third_dataset,
            '4': fourth_dataset,
            '5': center_dataset,
            }  

i3d = InceptionI3d(400, in_channels=3)
i3d.replace_logits(157)
i3d.load_state_dict(torch.load(load_model))
i3d.cuda()
i3d.train(False)  # Set model to evaluate mode
        
# Iterate over data.
for phase in ['1', '2', '3', '4', '5']:
    for data in dataloaders[phase]:
        # get the inputs
        inputs, labels, name = data
        b,c,t,h,w = inputs.shape
        features = []
        for start in range(1, t-16, 16):
            end = start + 16 
            ip = Variable(torch.from_numpy(inputs.numpy()[:,:,start:end]).cuda())
            features.append(i3d.extract_features(ip).squeeze(0).permute(1,2,3,0).data.cpu().numpy())
        # np.save(os.path.join(save_dir, name[0]+f"_{phase}"), np.concatenate(features, axis=0))
        np.save(os.path.join(save_dir, name[0]+f"_{phase}"), np.concatenate(features, axis=0).reshape(-1, 1024))
        print(f"Success Saved : {os.path.join(save_dir, name[0]+'_'+phase)}.npy")    

Success Saved : /opt/ml/input/data/test_video_1.npy
Success Saved : /opt/ml/input/data/test_video_2.npy
Success Saved : /opt/ml/input/data/test_video_3.npy
Success Saved : /opt/ml/input/data/test_video_4.npy
Success Saved : /opt/ml/input/data/test_video_5.npy


In [23]:
first_data = np.load('/opt/ml/input/data/test_video_1.npy')
second_data = np.load('/opt/ml/input/data/test_video_2.npy')
third_data = np.load('/opt/ml/input/data/test_video_3.npy')
fourth_data = np.load('/opt/ml/input/data/test_video_4.npy')
center_data = np.load('/opt/ml/input/data/test_video_5.npy')

print('first_crop :', first_data.shape)
print('second_crop :', second_data.shape)
print('third_crop :', third_data.shape)
print('fourth_crop :', fourth_data.shape)
print('center_crop :', center_data.shape)

first_crop : (510, 1024)
second_crop : (510, 1024)
third_crop : (510, 1024)
fourth_crop : (510, 1024)
center_crop : (510, 1024)
